<a href="https://colab.research.google.com/github/PhilSad/claire-instruct/blob/main/train_mixtral_claire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [ ]:
!pip install datasets huggingface_hub

In [6]:
import huggingface_hub
huggingface_hub.login()

In [2]:
import datasets

In [56]:
import random
def reduce_test(exemple):
  split = random.randint(20, 50)
  exemple['text'] = exemple['orig'][0:split]
  return exemple

In [57]:
dataset = datasets.load_dataset('OpenLLM-France/Claire-Dialogue-French-0.1')
dataset['test'] = dataset['test'].add_column("orig", dataset['test']['text'])
dataset['test'] = dataset['test'].map(reduce_test)
dataset['test'] = dataset['test'].remove_columns(['orig'])


Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Map:   0%|          | 0/284 [00:00<?, ? examples/s]

In [61]:
dataset.push_to_hub('PhilSad/Claire-Dialogue-French-0.1')

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/PhilSad/Claire-Dialogue-French-0.1/commit/cbb744b854b292a2fe17253ff2b582cbf9db23b8', commit_message='Upload dataset', commit_description='', oid='cbb744b854b292a2fe17253ff2b582cbf9db23b8', pr_url=None, pr_revision=None, pr_num=None)

In [53]:
ds.push_to_hub('philsad/Claire-Dialogue-French-0.1')

AttributeError: 'dict' object has no attribute 'push_to_hub'

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1",
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             )


In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

tokenizer.pad_token = "!" #Not EOS, will explain another time.\

CUTOFF_LEN = 32000  #Our dataset has shot text
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[ "w1", "w2", "w3"],  #just targetting the MoE layers.
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


In [ ]:
dataset = load_dataset("PhilSad/Claire-Dialogue-French-0.1")
print("dataset", dataset)
train_data = dataset["train"]
test_data = dataset["test"]


def generate_prompt(train_row):
    return train_row['text']

def tokenize(prompt):
    return tokenizer(
        prompt + tokenizer.eos_token,
        truncation=True,
        max_length=CUTOFF_LEN ,
        padding="max_length"
    )

train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))
test_data = test_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))


In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data
    args=TrainingArguments(
        report_to="wandb",
        bf16=True,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=1e-5,
        optim="adamw_torch",
        save_strategy="epoch",
        output_dir="mixtral-lora-claire",
        push_to_hub=True,
        save_total_limit=5,
        logging_strategy="steps",
        evaluation_strategy="steps",
        eval_steps = 30,
        logging_steps=1,
        save_steps = 30

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False



In [ ]:

trainer.train()